# LLM Sentiment analisis

In [3]:
# python 3.13.3
# !pip install openai==1.3.9
import openai
print(openai.__version__)

1.86.0


In [4]:
#ติดตั้งไลบรารีที่จำเป็น
# !pip install openai pandas tqdm scikit-learn

# นำเข้าไลบรารี
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",  
)

MODEL = "scb10x/llama3.1-typhoon2-70b-instruct"

# โหลดข้อมูล
df = pd.read_excel(r'C:\Users\FOth-PC\Desktop\เอกสาร\Ruby-test\rawdata_sentiment.xlsx') 

# ถ้า column ชื่ออื่น เปลี่ยนชื่อให้ตรง เช่น df.rename(columns={'Review': 'review', 'Label': 'label'}, inplace=True)
# df.rename(columns={'Text': 'review', 'Sentiment': 'label'}, inplace=True)

train_df, valid_df = train_test_split(
    df, test_size=0.2, stratify=df['Sentiment'], random_state=0
)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
print('valid_df size', valid_df.size)
df.head()

valid_df size 1200


,Text,Sentiment
0,นี้ๆชวนบ่อยสะ,neu
1,"(พยาบาลพัฒนาระบบคุณภาพ), โรงพยาบาลกรุงเทพภูเก็...",neu
2,เคยเจอแบบนี้ตอนนั่งแท็กซี่กลับจากคอนเสิร์ตคนเด...,neu
3,นิสสันมาแหะ,neu
4,อยากกินอ่ะหมู 😣😣😣,pos


In [5]:
# 5. สร้างฟังก์ชันเรียก OpenRouter LLM
def classify_sentiment(review, retry=2):
    prompt = f"""Classify the following review as Positive, Neutral, or Negative. Only output the label.

Review: "{review}"
Sentiment:"""
    for _ in range(retry):
        try:
            completion = client.chat.completions.create(
                model=MODEL,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=3
            )
            return completion.choices[0].message.content.strip()
        except Exception as e:
            print("Error:", e)
            time.sleep(2)
    return "error"


In [6]:
#few-shot prompt
import numpy as np

label_names = ['neu', 'pos', 'neg']
n_per_label = 1
fewshot_list = []

for label in label_names:
    sample = train_df[train_df['Sentiment'] == label].sample(n=n_per_label, random_state=42)
    for _, row in sample.iterrows():
        fewshot_list.append(f'Review: "{row["Text"]}" Label: {row["Sentiment"]}')

fewshot_prompt = '\n'.join(fewshot_list)

def ask_llm_with_fewshot(review, fewshot_prompt, model=MODEL, max_retry=2):
    prompt = f"""{fewshot_prompt}
Review: "{review}"
Label:"""
    for _ in range(max_retry):
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=3
            )
            return completion.choices[0].message.content.strip().lower()
        except Exception as e:
            print("Error:", e)
            time.sleep(2)
    return "error"



In [7]:
from tqdm import tqdm

valid_part = valid_df.sample(n=200, random_state=42)  # หรือจะวนทุกแถวก็ได้ (แต่ช้า)
results = []
for review in tqdm(valid_part["Text"]):
    results.append(ask_llm_with_fewshot(review, fewshot_prompt))

valid_part['llm_pred'] = results

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_true = valid_part["Sentiment"].str.lower()
y_pred = valid_part['llm_pred']

print("\nAccuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, digits=3))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred, labels=label_names))

100%|██████████| 200/200 [01:56<00:00,  1.72it/s]


Accuracy: 0.72
              precision    recall  f1-score   support

         neg      0.909     0.392     0.548        51
         neu      0.690     0.939     0.796       114
         pos      0.739     0.486     0.586        35

    accuracy                          0.720       200
   macro avg      0.780     0.605     0.643       200
weighted avg      0.755     0.720     0.696       200

Confusion matrix:
 [[107   6   1]
 [ 17  17   1]
 [ 31   0  20]]


In [8]:
valid_part.to_csv("llm_valid_with_pred.csv", index=False, encoding="utf-8-sig")